In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Logistic Regression

For reproducibility

In [3]:
#난수 생성을위한 시드를 설정. torch.Generator 객체를 반환
torch.manual_seed(1)

Training Data

In [4]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

In [5]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [6]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


Computing the Hypothesis (with torch.exp())

In [7]:
print('e^1 equals : ',torch.exp(torch.FloatTensor([1])))

e^1 equals :  tensor([2.7183])


In [10]:
W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [11]:
hypothesis = 1 / (1+torch.exp(-(x_train.matmul(W)+b)))

In [12]:
print(hypothesis)
print(hypothesis.shape)  #왜 이렇게 나오지??

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


Computing the Hypothesis(with torch.sigmoid())

In [13]:
print('e^1 equals : ',torch.sigmoid(torch.FloatTensor([1])))

e^1 equals :  tensor([0.7311])


In [14]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)

In [15]:
print(hypothesis)
print(hypothesis.shape)  #왜 이렇게 나오지??

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


Computing the Cost Function (log식)

In [16]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [18]:
-(y_train[0] * torch.log(hypothesis[0])+
   (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward>)

In [20]:
losses = -(y_train * torch.log(hypothesis)+
            (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [21]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


Computing the Cost Function (한번에)

In [22]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

### Whole Training Procedure

In [26]:
#모델 초기화
W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

#optimizer 설정
optimizer = torch.optim.SGD([W,b],lr=1)

nb_epochs =1000
for epoch in range(nb_epochs + 1):
    
    #cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W)+b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 ==0 : 
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch,nb_epochs,cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


Evaluation

In [27]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [28]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [29]:
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [30]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


### Higher Implementation with Class

In [32]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8,1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x):
        return self.sigmoid(self.linear(x))

In [33]:
model = BinaryClassifier()

In [35]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 1)

nb_epochs =1000
for epoch in range(nb_epochs + 1):
    
    #H(x) 계산
    hypothesis = model(x_train)
    
    #cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #20번마다 로그 출력
    if epoch % 10 ==0 : 
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch,nb_epochs,cost.item(),accuracy * 100
        ))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6x2 and 8x1)